In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import pandas as pd
from PIL import ImageDraw, ImageFont, Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torch.nn as nn
from transformers import ViTModel
from torchinfo import summary  # 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import random
import time
from sklearn.utils.class_weight import compute_class_weight
from tqdm.notebook import tqdm

In [2]:
DEVICE="cuda:0"
def setAllSeeds(seed):
  os.environ['MY_GLOBAL_SEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
# setAllSeeds(42)

In [3]:
df = pd.read_csv("train.csv")
categories=df["Category"].unique()
print(categories)
category=categories[1]
df = df[df["Category"]==category]
df=df.iloc[:500]

['Men Tshirts' 'Sarees' 'Kurtis' 'Women Tshirts' 'Women Tops & Tunics']


In [4]:
df.isna().sum()

id            0
Category      0
len           0
attr_1      274
attr_2       20
attr_3       66
attr_4       15
attr_5       16
attr_6      353
attr_7      255
attr_8       49
attr_9      119
attr_10      17
dtype: int64

In [5]:
i=1
df.iloc[:,3:2+i]=np.nan
df.iloc[:,3+i:]=np.nan
# df.iloc[:,[5,6]]=np.nan
print(df.head())


        id Category  len         attr_1 attr_2 attr_3 attr_4 attr_5 attr_6  \
7267  7432   Sarees   10  same as saree    NaN    NaN    NaN    NaN    NaN   
7268  7433   Sarees   10            NaN    NaN    NaN    NaN    NaN    NaN   
7269  7434   Sarees   10            NaN    NaN    NaN    NaN    NaN    NaN   
7270  7435   Sarees   10  same as saree    NaN    NaN    NaN    NaN    NaN   
7271  7436   Sarees   10          solid    NaN    NaN    NaN    NaN    NaN   

     attr_7 attr_8 attr_9 attr_10  
7267    NaN    NaN    NaN     NaN  
7268    NaN    NaN    NaN     NaN  
7269    NaN    NaN    NaN     NaN  
7270    NaN    NaN    NaN     NaN  
7271    NaN    NaN    NaN     NaN  


In [6]:
print(df.value_counts())

Series([], Name: count, dtype: int64)


In [7]:
delCol = []
idxCol = []
trackNum = []
weights=[]
m=nn.Softmax(dim=1)
l=[]
for i in range(1,11):
    uniName = df["attr_"+str(i)].dropna().unique()
    print(uniName)
    if(len(uniName)==0):
        delCol.append("attr_"+str(i))
    else:
        idxCol.append("attr_"+str(i))
        l.append('attr_'+str(i))
        trackNum.append(len(uniName))
        class_weights=compute_class_weight(class_weight="balanced",classes=uniName,y=df["attr_"+str(i)].dropna())
        weights.append(torch.tensor([1]*len(uniName),dtype=torch.float32).to(DEVICE))


['same as saree' 'solid' 'same as border' 'default']
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [8]:
df = df.drop(delCol,axis=1)
df.shape

(500, 4)

In [9]:
df.isna().sum()

id            0
Category      0
len           0
attr_1      274
dtype: int64

In [10]:
# df.dropna(inplace=True)

In [11]:
id2label={}
label2id={}
attrs={}
total_attr=len(df.columns)
for i in range(3,total_attr):
    labels=df[df.columns[i]].dropna().unique()
    # print(df.columns[i],labels)
    id2label[i-3]={k:labels[k] for k in range(len(labels))}
    label2id[i-3]={labels[k]:k for k in range(len(labels))}
    attrs[i-3]=df.columns[i]
print(id2label)
print(label2id)
print(attrs)

{0: {0: 'same as saree', 1: 'solid', 2: 'same as border', 3: 'default'}}
{0: {'same as saree': 0, 'solid': 1, 'same as border': 2, 'default': 3}}
{0: 'attr_1'}


In [12]:
def categorize(example):
    for i in attrs:
        # print(example[attrs[i]],type(example[attrs[i]]),pd.isna(example[attrs[i]]))
        if not pd.isna(example[attrs[i]]):
            example[attrs[i]]=label2id[i][example[attrs[i]]]
        else:
            example[attrs[i]]=''
    return example
df=df.apply(categorize,axis=1)
df.head()

,id,Category,len,attr_1
7267,7432,Sarees,10,0
7268,7433,Sarees,10,
7269,7434,Sarees,10,
7270,7435,Sarees,10,0
7271,7436,Sarees,10,1


In [13]:
df.isna().sum()

id          0
Category    0
len         0
attr_1      0
dtype: int64

In [14]:
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
# train_df=train_df.head(10)
# val_df,test_df=train_test_split(val_df,test_size=0.33)
# train_df=df
# val_df=train_df
# test_df=train_df

In [15]:
for i in l:
    print(train_df[i].value_counts())

attr_1
     220
0    112
2     37
3     29
1      2
Name: count, dtype: int64


In [16]:
# for i in l:
#     print(i)
#     for j in set(train_df[i].unique()):
#         x=train_df[i][train_df[i]==j]
#         if x.count()<300 and j!=-100:
#             y=train_df[train_df[i]==j]
#             y.iloc[:,3:]=-100
#             y[i]=j
#             print(y)
#             train_df=pd.concat([train_df,y,y]).reset_index(drop=True)
# for i in l:
#     print(train_df[i].value_counts())

In [17]:
train_df.head()

,id,Category,len,attr_1
7516,7681,Sarees,10,
7700,7865,Sarees,10,0
7286,7451,Sarees,10,0
7589,7754,Sarees,10,
7599,7764,Sarees,10,2


In [18]:
from transformers import AutoImageProcessor,BlipProcessor,BlipForQuestionAnswering
# model_name = 'google/vit-base-patch16-224'
model_name="Salesforce/blip-vqa-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForQuestionAnswering.from_pretrained(model_name).to("cuda")

In [19]:
class CustomFashionManager(Dataset):
    def __init__(self,csv_file, root_dir="./",transforms =None):
        self.fashionItems = csv_file
        self.root_dir = root_dir
        self.transforms = transforms
        self.category_list=pd.read_csv("categories.csv")

    def __len__(self):
        return len(self.fashionItems)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,f"{self.fashionItems.iloc[idx, 0]:06d}"+'.jpg')
        image = Image.open(img_name)
        # prompt=self.category_list[self.category_list["Category"]==self.fashionItems.iloc[idx, 1]].values()
        prompt="blouse_pattern"
        
        attributes = self.fashionItems.iloc[idx, 3]
        # attributes = np.array(attributes)
        # attributes = attributes.astype('float')
        # print(' '.join(attributes))
        inputs = processor(images=image, text=prompt, return_tensors="pt",padding="max_length",truncation=True)
        labels = processor(text=str(attributes),padding='max_length',max_length=10,return_tensors="pt").input_ids

        inputs["labels"] = labels
        # inputs['labels']=torch.tensor(attributes, dtype=torch.long)
        if self.transforms:
            inputs['pixel_values'] = self.transforms(inputs['pixel_values'])
            
        for k,v in inputs.items():  inputs[k] = v.squeeze()
        return inputs


In [20]:
from torchvision import transforms
transformations=transforms.Compose([
    transforms.RandomRotation(degrees=90),
    transforms.GaussianBlur(kernel_size=(9, 9), sigma=(0.1, 2.0)),
])
transformations=None
train_fashion_data = CustomFashionManager(csv_file=train_df,
                                    root_dir='train_images')
val_fashion_data = CustomFashionManager(csv_file=val_df,
                                    root_dir='train_images')
# test_fashion_data = CustomFashionManager(csv_file=test_df,root_dir='train_images')
        

In [21]:
from transformers import Trainer, TrainingArguments, TrainerCallback
from sklearn.metrics import classification_report,f1_score
batch_size = 32
# def collate_fn(batch):
    # return {
    #     'input_ids': torch.cat([x['input_ids'] for x in batch],dim=0),
    #     'attention_mask': torch.cat([x['attention_mask'] for x in batch],dim=0),
    #     'pixel_values': torch.cat([x['pixel_values'] for x in batch],dim=0),
    #     'labels': torch.cat([x['labels'] for x in batch],dim=0)
    # }

def compute_metrics(probs,labels):
    # probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    # probs=probs.T
    f1s=[]
    # for i in range(len(labels)):
    non_padding_indices = [j for j, label in enumerate(labels) if (label!= '' or label!=' ')]
    labels_ = [labels[j] for j in non_padding_indices]
    probs_ = [probs[j] for j in non_padding_indices]
    print(labels_)
    micro=f1_score(labels_,probs_,average='micro')
    macro=f1_score(labels_,probs_,average='macro')
    # print(f"attr_{i} f1 score: {macro}")
    print(classification_report(labels_,probs_))
    score=2*(micro*macro)/(micro+macro)
    f1s.append(score)
    print()
    print()
    return {'score': sum(f1s)/len(f1s)}


class CustomCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, model, tokenizer, eval_dataloader,**kwargs):
        # print("Evaluation done")
        model.config.use_cache = True
        val_loader=DataLoader(val_fashion_data,batch_size=1)
        preds=[]
        labels=[]
        for batch in tqdm(val_loader,total=len(val_loader)):
            input_ids=batch['input_ids'].to(DEVICE)
            pixel_values=batch['pixel_values'].to(DEVICE)
            attention_mask=batch['attention_mask'].to(DEVICE)
            outputs=model.generate(input_ids=input_ids,pixel_values=pixel_values,attention_mask=attention_mask)
            text = tokenizer.batch_decode(outputs,skip_special_tokens=True)
            label=tokenizer.batch_decode(batch['labels'],skip_special_tokens=True)
            preds.extend(text)
            labels.extend(label)
        print(compute_metrics(preds,labels))


training_args = TrainingArguments(
  output_dir="./blip/"+category,
  per_device_train_batch_size=10,
  per_device_eval_batch_size=10,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=10,
  fp16=True,
  learning_rate=2e-5,
#   dataloader_num_workers=20,lr

  save_total_limit=1,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
#   metric_for_best_model="score",
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_fashion_data,
    eval_dataset=val_fashion_data,
    # data_collator=collate_fn,
    # compute_metrics=compute_metrics,
    tokenizer=processor,
)
trainer.add_callback(CustomCallback())

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
trainer.train()
trainer.save_model(f"./blip/{category}/final")

[2024-11-10 20:09:54,564] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/aseems/anaconda3/envs/mhcp4/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


...

  0%|          | 0/100 [00:00<?, ?it/s]

['0', '2', '', '0', '0', '', '', '', '2', '', '', '0', '', '0', '', '', '', '', '', '', '0', '1', '0', '0', '', '', '0', '', '0', '0', '', '2', '0', '1', '', '0', '3', '0', '', '', '', '', '', '0', '0', '0', '0', '0', '', '0', '3', '', '0', '', '', '1', '', '', '', '0', '', '', '0', '', '', '', '', '0', '', '', '', '', '', '', '0', '2', '', '', '0', '', '', '0', '0', '', '', '0', '2', '', '0', '', '0', '', '0', '', '', '2', '0', '0', '2', '']
              precision    recall  f1-score   support

                   0.00      0.00      0.00        54
           0       0.34      1.00      0.51        34
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00         2

    accuracy                           0.34       100
   macro avg       0.07      0.20      0.10       100
weighted avg       0.12      0.34      0.17       100



{'score': 0.15632183908045977}


  0%|          | 0/100 [00:00<?, ?it/s]

['0', '2', '', '0', '0', '', '', '', '2', '', '', '0', '', '0', '', '', '', '', '', '', '0', '1', '0', '0', '', '', '0', '', '0', '0', '', '2', '0', '1', '', '0', '3', '0', '', '', '', '', '', '0', '0', '0', '0', '0', '', '0', '3', '', '0', '', '', '1', '', '', '', '0', '', '', '0', '', '', '', '', '0', '', '', '', '', '', '', '0', '2', '', '', '0', '', '', '0', '0', '', '', '0', '2', '', '0', '', '0', '', '0', '', '', '2', '0', '0', '2', '']
              precision    recall  f1-score   support

                   0.00      0.00      0.00        54
           0       0.34      1.00      0.51        34
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00         2

    accuracy                           0.34       100
   macro avg       0.07      0.20      0.10       100
weighted avg       0.12      0.34      0.17       100



{'score': 0.15632183908045977}


KeyboardInterrupt: 

In [ ]:
trainer.evaluate(val_fashion_data)
#1-54(imbalance),2-65,3-85,5-56,6-70,7-52(imbalance),8-51(imbalance),9-60(imbalance),10-49(imbalance)
#4-30,

  0%|          | 0/20 [00:00<?, ?it/s]

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['', '0', '0', '', '0', '', '0', '', '0', '0', '', '', '2', '0', '0', '1', '', '0', '', '0']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
['', '0', '0', '', '0', '', '0', '', '0', '0', '', '', '2', '0', '0', '1', '', '0', '', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
              precision    recall  f1-score   support

                   0.00      0.00      0.00         8
           0       0.50      1.00      0.67        10
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1

    accuracy                           0.50        20
   macro avg       0.12      0.25      0.17        20
weighted avg       0.25      0.50      0.33        20



{'score': 0.25}


{'eval_loss': 6.505589485168457,
 'eval_runtime': 3.1286,
 'eval_samples_per_second': 6.393,
 'eval_steps_per_second': 6.393,
 'epoch': 10.0}